# 4. Multiphase equilibrium with brine
<img style = "float: left;" src = "slides/3a_negative_flash.png" width = "60%">

## 4.1. Solubility of brine and CO$_2$ in different phases

In [ ]:
import numpy as np
import xarray as xr
from dartsflash.pyflash import PyFlash
from dartsflash.mixtures import Mixture, ConcentrationUnits
from dartsflash.libflash import CubicEoS, AQEoS, InitialGuess


mix = Mixture(components=["H2O", "CO2"], ions=["Na+", "Cl-"])
f = PyFlash(mixture=mix)

f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.PR),
          initial_guesses=[InitialGuess.Yi.Wilson])
f.add_eos("AQ", AQEoS(mix.comp_data, {AQEoS.water: AQEoS.Jager2003,
                                      AQEoS.solute: AQEoS.Ziabakhsh2012,
                                      AQEoS.ion: AQEoS.Jager2003}))
f.init_flash(stabilityflash=False, eos_order=["AQ", "CEOS"], initial_guess=[InitialGuess.Henry_AV], np_max=2)

state_spec = {"pressure": np.arange(1., 600., 1.),
              "temperature": np.array([25, 90]) + 273.15,
             }
compositions = {"H2O": 0.8,
                "CO2": 0.2
               }
results_m0 = f.evaluate_flash(state_spec=state_spec, compositions=compositions,
                              mole_fractions=True, concentrations={"NaCl": 0.})
results_m2 = f.evaluate_flash(state_spec=state_spec, compositions=compositions,
                              mole_fractions=True, concentrations={"NaCl": 2.})
flash_results = xr.concat([results_m0, results_m2], dim='concentrations')


In [ ]:
from dartsflash.plot import PlotFlash
labels = ['25$\degree$C, m=0', '90$\degree$C, m=0', '25$\degree$C, m=2', '90$\degree$C, m=2']

plotx = PlotFlash.solubility(f, flash_results, dissolved_comp_idx=1, phase_idx=0,
                             labels=labels, concentrations=[0, 2])

ploty = PlotFlash.solubility(f, flash_results, dissolved_comp_idx=0, phase_idx=1,
                             labels=labels, concentrations=[0, 2])
ploty.ax[0].set(ylim=[0., 0.04])

## 4.2. Binary gas mixture with water

In [ ]:
import numpy as np
import xarray as xr
from dartsflash.libflash import FlashParams, EoSParams, EoS, InitialGuess
from dartsflash.libflash import CubicEoS, AQEoS

# H2O-CO2-C1 mixtures
components = ["H2O", "CO2", "C1"]

mix = Mixture(components, setprops=True)
f = PyFlash(mix)

f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.PR),
          initial_guesses=[0, 1, 2], preferred_roots=[(0, 0.75, EoS.MAX)],
          switch_tol=1e-1, stability_tol=1e-20, max_iter=50, use_gmix=0)
f.add_eos("AQ", AQEoS(mix.comp_data, {AQEoS.water: AQEoS.Jager2003,
                                      AQEoS.solute: AQEoS.Ziabakhsh2012}),
          initial_guesses=[0], eos_range={0: [0.6, 1.]}, max_iter=10, use_gmix=True)

f.flash_params.stability_variables = FlashParams.alpha
f.flash_params.split_variables = FlashParams.nik
f.flash_params.split_tol = 1e-20
# f.flash_params.split_switch_tol = 1e-2
f.flash_params.tpd_close_to_boundary = 1e-2
f.flash_params.split_negative_flash_iter = 3
f.flash_params.tpd_tol = 1e-11
f.flash_params.verbose = 0

eos_order = ["AQ", "CEOS"]
np_max = 3
f.init_flash(eos_order=eos_order, stabilityflash=True, np_max=np_max)

In [ ]:
""" TERNARY DIAGRAM """
state_spec = {"pressure": 100.,
              "temperature": 375.,
             }

dz = 0.0001
min_z = [0.97, 0.]
max_z = [1., 0.03]
compositions = {comp: np.arange(min_z[i], max_z[i]+0.1*dz, dz) for i, comp in enumerate(components[:-1])}
compositions[components[-1]] = 1.

x0 = compositions[components[0]]
x1 = compositions[components[1]]

flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True, print_state="Flash")

plot = PlotFlash.ternary(f, flash_results, state=state_spec, dz=dz, min_z=min_z, max_z=max_z, plot_phase_fractions=False)

## 4.3. Hydrates formation

<img style = "float: left;" src = "slides/3c_vdwp.png" width = "60%">

In [ ]:
from dartsflash.hyflash import HyFlash
from dartsflash.mixtures import Mixture, ConcentrationUnits as cu
from dartsflash.libflash import CubicEoS, AQEoS, Ballard, InitialGuess


mix = Mixture(components=["H2O", "CO2"], ions=["Na+", "Cl-"])
f = HyFlash(mixture=mix)

state_spec = {"temperature": np.arange(271., 291., 0.1),
              "pressure": None
             }
compositions = {"H2O": 0.8,
                "CO2": 0.2,
               }
concentrations = [{"NaCl": m} for m in [0., 0.01, 0.05, 0.10, 0.15]]  #, 0.20, 0.25]]
concentration_unit = cu.WEIGHT

labels = ["m = {:.0f} wt%".format(c["NaCl"]*100) for c in concentrations]

f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.SRK),
          initial_guesses=[InitialGuess.Yi.Wilson])
f.add_eos("AQ", AQEoS(mix.comp_data, {AQEoS.water: AQEoS.Jager2003,
                                      AQEoS.solute: AQEoS.Ziabakhsh2012,
                                      AQEoS.ion: AQEoS.Jager2003}))
f.add_hydrate_eos("sI", Ballard(mix.comp_data, "sI"))

f.flash_params.eos_order = ["AQ", "CEOS"]
f.init_flash(stabilityflash=False, eos_order=["AQ", "CEOS"], initial_guess=[InitialGuess.Henry_AV])

if concentrations is not None:
    results = []
    for concentration in concentrations:
        results.append(f.evaluate_equilibrium(state_spec, compositions, mole_fractions=True,
                                              concentrations=concentration, concentration_unit=concentration_unit))
    results = xr.concat(results, dim='concentrations')
else:
    results = f.evaluate_equilibrium(state_spec, compositions, mole_fractions=True, print_state="")

In [ ]:
from dartsflash.plot import PlotHydrate

PlotHydrate.pt(f, results, compositions_to_plot=compositions, concentrations=concentrations, logy=True,
               labels=labels, legend_loc="lower right")
